In [ ]:
import cv2
import numpy as np
from skimage.measure import regionprops

def mask_cartridge_case(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale for better processing
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and improve contour detection
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # Apply the Canny edge detection algorithm
    edges = cv2.Canny(blurred_image, 100, 200)

    # Morphological operations to clean up the edge map
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel)

    # Apply erosion to the edge map to remove small edges
    erosion = cv2.erode(opened, kernel, iterations=1)

    # Apply dilation to the eroded edge map to restore the edges that were removed
    dilation = cv2.dilate(erosion, kernel, iterations=1)

    # Find contours in the edge map
    contours, _ = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize lists to store the bounding boxes of the contours
    breech_face_contours = []
    aperture_shear_contours = []
    firing_pin_contours = []

    # Iterate through the contours and extract the bounding boxes
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > h:  # Breech-face impression is typically wider than it is tall
            breech_face_contours.append((x, y, w, h))
        elif w < h and w/h > 0.5:  # Aperture shear is typically taller than it is wide and has an aspect ratio greater than 0.5
            aperture_shear_contours.append((x, y, w, h))
        elif w == h:  # Firing pin impression is typically square
            firing_pin_contours.append((x, y, w, h))

    # Extract the regions of interest (ROIs) from the image using the bounding boxes
    if breech_face_contours:
        breech_face_roi = image[min(breech_face_contours)[1]:max(breech_face_contours)[1]+max(breech_face_contours)[3],
                                min(breech_face_contours)[0]:max(breech_face_contours)[0]+max(breech_face_contours)[2]]

    if aperture_shear_contours:
        aperture_shear_roi = image[min(aperture_shear_contours)[1]:max(aperture_shear_contours)[1]+max(aperture_shear_contours)[3],
                                   min(aperture_shear_contours)[0]:max(aperture_shear_contours)[0]+max(aperture_shear_contours)[2]]

    if firing_pin_contours:
        firing_pin_roi = image[min(firing_pin_contours)[1]:max(firing_pin_contours)[1]+max(firing_pin_contours)[3],
                               min(firing_pin_contours)[0]:max(firing_pin_contours)[0]+max(firing_pin_contours)[2]]

    # Convert the ROIs to grayscale and apply thresholding
    gray_breech_face = cv2.cvtColor(breech_face_roi, cv2.COLOR_BGR2GRAY)
    _, binary_breech_face = cv2.threshold(gray_breech_face, 127, 255, cv2.THRESH_BINARY)

    gray_aperture_shear = cv2.cvtColor(aperture_shear_roi, cv2.COLOR_BGR2GRAY)
    _, binary_aperture_shear = cv2.threshold(gray_aperture_shear, 127, 255, cv2.THRESH_BINARY)

    gray_firing_pin = cv2.cvtColor(firing_pin_roi, cv2.COLOR_BGR2GRAY)
    _, binary_firing_pin = cv2.threshold(gray_firing_pin, 127, 255, cv2.THRESH_BINARY)

    # Calculate the moments of the binary images
    props_breech_face = regionprops(binary_breech_face)
    props_aperture_shear = regionprops(binary_aperture_shear)
    props_firing_pin = regionprops(binary_firing_pin)

    # Calculate the orientation of the ROIs
    breech_face_orientation = props_breech_face[0].orientation
    aperture_shear_orientation = props_aperture_shear[0].orientation
    firing_pin_orientation = props_firing_pin[0].orientation

    # Calculate the direction of the firing pin drag
    firing_pin_drag_direction = aperture_shear_orientation - firing_pin_orientation

    # Analyze the shape and size of the ROIs
    breech_face_perimeter = props_breech_face[0].perimeter
    breech_face_aspect_ratio = props_breech_face[0].minor_axis_length / props_breech_face[0].major_axis_length
    breech_face_circularity = (4 * np.pi * props_breech_face[0].area) / (breech_face_perimeter ** 2)

    aperture_shear_perimeter = props_aperture_shear[0].perimeter
    aperture_shear_aspect_ratio = props_aperture_shear[0].minor_axis_length / props_aperture_shear[0].major_axis_length
    aperture_shear_circularity = (4 * np.pi * props_aperture_shear[0].area) / (aperture_shear_perimeter ** 2)

    firing_pin_perimeter = props_firing_pin[0].perimeter
    firing_pin_aspect_ratio = props_firing_pin[0].minor_axis_length / props_firing_pin[0].major_axis_length
    firing_pin_circularity = (4 * np.pi * props_firing_pin[0].area) / (firing_pin_perimeter ** 2)

    # Initialize masks with the shape of the image
    breech_face_mask = np.zeros_like(image)
    aperture_shear_mask = np.zeros_like(image)
    firing_pin_mask = np.zeros_like(image)

    # Convert bounding boxes to contours
    breech_face_contours = [np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]]) for x, y, w, h in breech_face_contours]
    aperture_shear_contours = [np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]]) for x, y, w, h in aperture_shear_contours]
    firing_pin_contours = [np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]]) for x, y, w, h in firing_pin_contours]

    # Draw contours on masks
    if breech_face_contours:
        cv2.drawContours(breech_face_mask, breech_face_contours, -1, (255, 0, 0), -1)

    if aperture_shear_contours:
        cv2.drawContours(aperture_shear_mask, aperture_shear_contours, -1, (0, 255, 0), -1)

    if firing_pin_contours:
        cv2.drawContours(firing_pin_mask, firing_pin_contours, -1, (128, 0, 128), -1)

    # Combine masks
    combined_mask = breech_face_mask + aperture_shear_mask + firing_pin_mask

    # Convert combined_mask to a single-channel (grayscale) mask
    combined_mask_gray = cv2.cvtColor(combined_mask, cv2.COLOR_BGR2GRAY)

    # Apply the combined mask to the original image
    result_image = cv2.bitwise_and(image, image, mask=combined_mask_gray)

    # Add an arrow indicating the direction of firing pin drag
    arrow_length = 50
    arrow_angle = 30
    arrow_tip = (int(image.shape[1] / 2), int(image.shape[0] / 2))
    arrow_end = (
        int(arrow_tip[0] + arrow_length * np.cos(np.radians(arrow_angle))),
        int(arrow_tip[1] - arrow_length * np.sin(np.radians(arrow_angle)))
    )
    cv2.arrowedLine(result_image, arrow_tip, arrow_end, (255, 0, 0), thickness=2)  # Blue arrow

    # Display the original image and the masked result
    cv2.imshow('Original Image', image)
    cv2.imshow('Masked Result', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
image_path = "image_0.jpg"
mask_cartridge_case(image_path)


Here I have used a single image for example because there was no dataset available on cartridge cases. And I didn't create a dataset because the images availble on internet were of varied quality (size, pixels etc.)